In [ ]:
import os
import random
import warnings
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
import torch.optim as optim
from einops import rearrange
from tqdm import tqdm
from torchvision.transforms import GaussianBlur
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, mean_squared_error, f1_score, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from scipy.stats import gaussian_kde
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler

# Device configuration and seeding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
class Augmentation:
    def __init__(self, size=112, crop_scale=(0.8, 1.0), brightness=(0.8, 1.2)):
        self.size = size
        self.crop_scale = crop_scale
        self.brightness = brightness

    def __call__(self, x):
        # x is a torch tensor image of shape (C, H, W)
        _, h, w = x.shape
        
        # Random Resized Crop
        scale = random.uniform(self.crop_scale[0], self.crop_scale[1])
        new_h = int(h * scale)
        new_w = int(w * scale)
        if h > new_h and w > new_w:
            top = random.randint(0, h - new_h)
            left = random.randint(0, w - new_w)
            x = x[:, top:top+new_h, left:left+new_w]
        # Resize to the target size
        x = F.interpolate(x.unsqueeze(0), size=(self.size, self.size),
                          mode='bilinear', align_corners=False).squeeze(0)
        
        # Random Horizontal Flip
        if random.random() < 0.5:
            x = torch.flip(x, dims=[2])
            
        # Random brightness jitter
        if random.random() < 0.5:
            factor = random.uniform(self.brightness[0], self.brightness[1])
            x = x * factor
            x = torch.clamp(x, 0, 1)
        return x

In [ ]:

class JetDatasetSSL(Dataset):
    def __init__(self, file_path, key, transform=None):
        self.file_path = file_path
        self.key = key
        with h5py.File(file_path, 'r') as f:
            self.length = f[key].shape[0]
        self.transform = transforms.Compose([Augmentation()]) if transform is None else transform

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.file_path, 'r') as f:
            jet = f[self.key][idx]  # Expected shape: [125, 125, 8]
        # Convert shape to [channels, eta, phi]
        jet = torch.tensor(jet, dtype=torch.float32).permute(2, 0, 1)
        # Create two augmented views for contrastive learning
        view1 = self.transform(jet)
        view2 = self.transform(jet)
        return view1, view2

In [5]:
# New class for labeled dataset
class JetDatasetLabeled(Dataset):
    def __init__(self, file_path, jet_key="jet", y_key="Y", pt_key="pT", m_key="m"):
        self.file_path = file_path
        self.jet_key = jet_key
        self.y_key = y_key
        self.pt_key = pt_key
        self.m_key = m_key
        
        with h5py.File(file_path, 'r') as f:
            self.length = f[jet_key].shape[0]
            
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.file_path, 'r') as f:
            jet = f[self.jet_key][idx]
            y = f[self.y_key][idx]
            pt = f[self.pt_key][idx]
            m = f[self.m_key][idx]
            
        # Convert shape to [channels, eta, phi]
        jet = torch.tensor(jet, dtype=torch.float32).permute(2, 0, 1)
        y = torch.tensor(y, dtype=torch.long)
        pt = torch.tensor(pt, dtype=torch.float32)
        m = torch.tensor(m, dtype=torch.float32)
        
        return jet, y, pt, m

In [ ]:
class ResNetSSL(nn.Module):
    def __init__(self, latent_dim=256, projection_dim=128, resnet_depth=18):
        super().__init__()

        if resnet_depth == 18:
            self.encoder_backbone = models.resnet18(weights=None)
            num_bottleneck_features = 512
        elif resnet_depth == 34:
            self.encoder_backbone = models.resnet34(weights=None)
            num_bottleneck_features = 512
        else:
            raise ValueError("Unsupported ResNet depth")

        original_conv1 = self.encoder_backbone.conv1
        self.encoder_backbone.conv1 = nn.Conv2d(
            8,
            original_conv1.out_channels,
            kernel_size=original_conv1.kernel_size,
            stride=original_conv1.stride,
            padding=original_conv1.padding,
            bias=False
        )

        self.encoder_backbone.fc = nn.Identity()

        self.final_layer = nn.Linear(num_bottleneck_features, latent_dim)

        self.projector = nn.Sequential(
            nn.Linear(latent_dim, latent_dim * 2),
            nn.ReLU(),
            nn.Linear(latent_dim * 2, projection_dim)
        )

    def forward(self, x):
        features_bottleneck = self.encoder_backbone(x)
        features_latent = self.final_layer(features_bottleneck)
        projections = self.projector(features_latent)
        return features_latent, projections


In [7]:
class LARS(optim.Optimizer):
    def __init__(self, optimizer, eps=1e-9, trust_coef=0.002):
        """
        LARS wrapper for an optimizer with updated hyperparameters.

        Args:
            optimizer: The inner optimizer (e.g. AdamW).
            eps: Small value for numerical stability.
            trust_coef: Coefficient for computing the local learning rate.
        """
        self.optimizer = optimizer
        self.eps = eps
        self.trust_coef = trust_coef

    def step(self, closure=None):
        for group in self.optimizer.param_groups:
            weight_decay = group.get("weight_decay", 0)
            for p in group["params"]:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p = d_p.add(p.data, alpha=weight_decay)
                w_norm = p.data.norm()
                g_norm = d_p.norm()
                if w_norm > 0 and g_norm > 0:
                    local_lr = self.trust_coef * w_norm / (g_norm + self.eps)
                    d_p.mul_(local_lr)
        self.optimizer.step(closure)

    def zero_grad(self):
        self.optimizer.zero_grad()

In [8]:
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.05, eps=1e-6):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature
        self.eps = eps

    def forward(self, z_i, z_j):
        batch_size = z_i.size(0)

        # Normalize the input embeddings
        z_i = F.normalize(z_i, dim=1, eps=self.eps)
        z_j = F.normalize(z_j, dim=1, eps=self.eps)

        # Concatenate the embeddings
        representations = torch.cat([z_i, z_j], dim=0)

        # Compute similarity matrix (use float32 for stability)
        similarity_matrix = torch.mm(representations, representations.t()) / self.temperature
        similarity_matrix = similarity_matrix.float()

        # Mask out self-similarity using a safer mask value
        mask = torch.eye(2 * batch_size, device=z_i.device).bool()
        similarity_matrix = similarity_matrix.masked_fill(mask, -1e4)

        # Compute positive and negative samples
        positives = torch.cat([
            torch.diag(similarity_matrix, batch_size),
            torch.diag(similarity_matrix, -batch_size)
        ])

        negatives = similarity_matrix[~mask].view(2 * batch_size, -1)

        # Calculate the loss using logsumexp for numerical stability
        logsumexp_negatives = torch.logsumexp(negatives, dim=1)
        loss = -torch.log(torch.exp(positives) / (torch.exp(logsumexp_negatives) + self.eps)).mean()

        return loss

In [9]:
def train_ssl(model, train_loader, num_epochs=100, save_dir="./models", validation_loader=None, patience=5, use_lars=True):
    """
    Train the SSL model with improved monitoring, validation, early stopping, and optional LARS.
    
    Args:
        model: The SimCLR model to train.
        train_loader: DataLoader for training data.
        num_epochs: Number of training epochs.
        save_dir: Directory to save model checkpoints.
        validation_loader: Optional loader for validation.
        patience: Number of epochs with no improvement after which training will be stopped.
        use_lars: If True, wraps the underlying optimizer with LARS.
        
    Returns:
        Trained model and training history.
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Setup base optimizer using AdamW
    base_optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=4e-5)
    optimizer = LARS(base_optimizer, eps=1e-9, trust_coef=0.002) if use_lars else base_optimizer
    
    # Use the inner optimizer for the scheduler if LARS wrapper is used
    scheduler = CosineAnnealingLR(optimizer.optimizer if use_lars else optimizer, T_max=num_epochs)
    
    # Contrastive loss with temperature scaling
    criterion = NTXentLoss(temperature=0.05)
    
    best_loss = float('inf')
    history = {'train_loss': [], 'val_loss': []}
    early_stop_counter = 0
    scaler = GradScaler(device=device) if device.type == "cuda" else GradScaler()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for view1, view2 in pbar:
            x1, x2 = view1.to(device), view2.to(device)
            optimizer.zero_grad()
            
            with autocast(device_type=device.type):
                _, proj1 = model(x1)
                _, proj2 = model(x2)
                loss = criterion(proj1, proj2)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer.optimizer if use_lars else optimizer)
            scaler.update()
            
            total_loss += loss.item()
            current_lr = (optimizer.optimizer.param_groups[0]['lr'] if use_lars 
                          else optimizer.param_groups[0]['lr'])
            pbar.set_postfix({'loss': loss.item(), 'lr': current_lr})
        
        avg_loss = total_loss / len(train_loader)
        history['train_loss'].append(avg_loss)
        
        current_loss = avg_loss
        if validation_loader is not None:
            model.eval()
            val_total_loss = 0.0
            with torch.no_grad():
                for view1, view2 in validation_loader:
                    x1, x2 = view1.to(device), view2.to(device)
                    _, proj1 = model(x1)
                    _, proj2 = model(x2)
                    loss = criterion(proj1, proj2)
                    val_total_loss += loss.item()
            val_loss = val_total_loss / len(validation_loader)
            history['val_loss'].append(val_loss)
            print(f"Epoch {epoch+1} - Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")
            current_loss = val_loss
        else:
            print(f"Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': (optimizer.optimizer.state_dict() if use_lars 
                                         else optimizer.state_dict()),
                'loss': best_loss,
                'history': history,
            }, os.path.join(save_dir, "best_model.pt"))
            print(f"New best model saved! Loss: {best_loss:.4f}")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1} with best loss {best_loss:.4f}")
                break
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': (optimizer.optimizer.state_dict() if use_lars 
                                     else optimizer.state_dict()),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': current_loss,
            'history': history,
        }, os.path.join(save_dir, f"checkpoint_epoch_{epoch+1}.pt"))
            
        scheduler.step()
    
    # Plot training curve
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    if history['val_loss']:
        plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.semilogy(history['train_loss'], label='Training Loss')
    if history['val_loss']:
        plt.semilogy(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.legend()
    plt.title('Loss on Log Scale')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "training_curve.png"))
    plt.close()
    
    return model, history

In [10]:

# Extract features using the pre-trained encoder
def extract_features(model, dataloader):
    model.eval()
    features_list = []
    y_list = []
    pt_list = []
    m_list = []
    
    with torch.no_grad():
        for jets, y, pt, m in tqdm(dataloader, desc="Extracting features"):
            jets = jets.to(device)
            features, _ = model(jets)
            features_list.append(features.cpu())
            y_list.append(y)
            pt_list.append(pt)
            m_list.append(m)
    
    # Concatenate all batches
    features = torch.cat(features_list, dim=0)
    y = torch.cat(y_list, dim=0)
    pt = torch.cat(pt_list, dim=0)
    m = torch.cat(m_list, dim=0)
    
    return features, y, pt, m



In [11]:

def main():
    # Configuration
    ssl_file_path = "/kaggle/input/dataset-specific-unlabelled/Dataset_Specific_Unlabelled.h5"
    labeled_file_path = "/kaggle/input/dataset-specific-labelled-full-only-for-2i/Dataset_Specific_labelled_full_only_for_2i.h5"
    batch_size = 128
    num_epochs_ssl = 30  # Increased for better convergence
    latent_dim = 256
    save_dir = "./models"
    os.makedirs(save_dir, exist_ok=True)
    
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    
    # Step 1: Self-Supervised Learning Pretraining
    print("Loading SSL dataset...")
    try:
        ssl_dataset = JetDatasetSSL(ssl_file_path, "jet")
        print(f"Dataset loaded with {len(ssl_dataset)} samples")
        
        # Create train/validation split for SSL
        train_size = int(0.9 * len(ssl_dataset))
        val_size = len(ssl_dataset) - train_size
        ssl_train_dataset, ssl_val_dataset = torch.utils.data.random_split(
            ssl_dataset, [train_size, val_size], 
            generator=torch.Generator().manual_seed(42)
        )
        
        ssl_train_loader = DataLoader(
            ssl_train_dataset, 
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            drop_last=True
        )
        
        ssl_val_loader = DataLoader(
            ssl_val_dataset, 
            batch_size=batch_size,
            shuffle=False,
            num_workers=4,
            pin_memory=True,
            drop_last=True
        )
        
        # Initialize model
        print(f"Initializing SimCLR model on {device}")
        ssl_model = model = ResNetSSL(latent_dim=256, projection_dim=128, resnet_depth=18).to(device)
        
        # Print model summary
        total_params = sum(p.numel() for p in ssl_model.parameters())
        print(f"Model created with {total_params:,} parameters")
        
        # Train model
        print("Starting SSL training...")
        ssl_model, history = train_ssl(
            ssl_model, 
            ssl_train_loader, 
            num_epochs=num_epochs_ssl, 
            save_dir=save_dir,
            validation_loader=ssl_val_loader ,use_lars=True)  
        
        print("SSL training completed!")
        
    except Exception as e:
        print(f"Error during SSL training: {e}")
        raise

if __name__ == "__main__":
    main()

Loading SSL dataset...
Dataset loaded with 60000 samples
Initializing SimCLR model on cuda
Model created with 11,520,768 parameters
Starting SSL training...


Epoch 1/30: 100%|██████████| 421/421 [03:39<00:00,  1.92it/s, loss=2.11, lr=0.0003]


Epoch 1 - Train Loss: 3.6512, Val Loss: 2.2338
New best model saved! Loss: 2.2338


Epoch 2/30: 100%|██████████| 421/421 [03:41<00:00,  1.90it/s, loss=0.795, lr=0.000299]


Epoch 2 - Train Loss: 1.3974, Val Loss: 0.7458
New best model saved! Loss: 0.7458


...

Epoch 26 - Train Loss: 0.0040, Val Loss: 0.0043


Epoch 27/30: 100%|██████████| 421/421 [03:37<00:00,  1.94it/s, loss=0.0162, lr=1.3e-5]


Epoch 27 - Train Loss: 0.0041, Val Loss: 0.0045


Epoch 28/30: 100%|██████████| 421/421 [03:35<00:00,  1.95it/s, loss=0.00334, lr=7.34e-6]


Epoch 28 - Train Loss: 0.0040, Val Loss: 0.0039
Early stopping triggered at epoch 28 with best loss 0.0037
SSL training completed!
